In [ ]:
%%writefile final_project.py
#!usr/bin/env python
import os
import sklearn as skl
from sklearn.model_selection import GridSearchCV, LeaveOneOut, cross_validate, cross_val_score, KFold, train_test_split
from sklearn.metrics import SCORERS
from sklearn import metrics
 
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

import numpy as np
from scipy.stats import pearsonr, spearmanr
import itertools

# make sure the experiment can be replicated.
seed = 42
k10 = KFold(10, shuffle=True, random_state=seed)

# download and load data
os.system('rm -f organic_chemistry_data.csv && wget https://raw.githubusercontent.com/forrestbao/MLClass/master/projects/organic_chemistry_data.csv')

f = open('organic_chemistry_data.csv', 'rb')
data = np.loadtxt(f, delimiter=',', skiprows=1)
f.close()

x = data[:, [2] + list(range(8, 12))]
y = data[:, 3: 8]

# preprocess data
scaler = preprocessing.StandardScaler().fit(x)
X = scaler.transform(x)

# split data to training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=seed)
print('Train size:', len(y_train), '; Test size:', len(y_test))
print('\n')

loo = LeaveOneOut()

###################################################################
# 1. train the models and find the best hyperparameters for each model by 10-fold or leaveoneout cv.
###################################################################
print('1. train the models and find the best hyperparameters for each model by 10-fold or leaveoneout cv.')
# 1.1. SVR
clf_svr = MultiOutputRegressor(SVR())
param_svr = {
    'estimator__C': [10**elem for elem in range(-5, 5)],
    'estimator__gamma': [10**elem for elem in range(-5, 5)]
}

gs_svr = (GridSearchCV(estimator=clf_svr, 
                        param_grid=param_svr, 
                        #cv=loo,
                        cv=KFold(10, shuffle=True, random_state=seed),
                        #scoring='neg_mean_squared_error',
                        scoring='r2',
                        n_jobs=-1))

print('\n1.1 SVR:')
print('\nhyperparameters and their ranges:')
#print(param_svr)
for i, j in param_svr.items():
    print(i[11:] + ':', j)

print('\nthe best hyperparameters:')
#gs_svr.fit(X_train, y_train)
gs_svr.fit(X, y)
#print((gs_svr.predict(X_test) - y_test))
#print(gs_svr.best_params_)
for i, j in gs_svr.best_params_.items():
    print('best ' + i[11:] + ':', j)


# 1.2. Randomforest
clf_rf = MultiOutputRegressor(RandomForestRegressor())
param_rf = {
    'estimator__max_depth': [1, 2, 4, 8, 16, 32],
    'estimator__min_impurity_decrease': [0, 1e-10, 1e-5]
}

gs_rf = (GridSearchCV(estimator=clf_rf, 
                        param_grid=param_rf, 
                        #cv=loo,
                        cv=KFold(10, shuffle=True, random_state=seed),
                        #scoring =['neg_mean_absolute_error', 'r2'],
                        #scoring='neg_mean_squared_error',
                        scoring='r2',
                        n_jobs=-1))

print('\n1.2 Random forest:')
print('\nhyperparameters and their ranges:')
#print(param_rf)
for i, j in param_rf.items():
    print(i[11:] + ':', j)

print('\nthe best hyperparameters:')
#gs_rf.fit(X_train, y_train)
gs_rf.fit(X, y)
#print((gs_rf.predict(X_test) - y_test))
#print(gs_rf.best_params_)
for i, j in gs_rf.best_params_.items():
    print('best ' + i[11:] + ':', j)

# 1.3. Nerual network

# generate the layer and neuron parameters.
lyrs = []
for i in range(3):
    lyrs.extend(itertools.product([16, 24, 32], repeat=i+1))

clf_nn = MultiOutputRegressor(MLPRegressor())
param_nn = {
    'estimator__hidden_layer_sizes': lyrs,
     'estimator__max_iter': [2, 8, 32, 128, 256],
     'estimator__alpha': [1e-5, 1e-3, 1e-1]
}

gs_nn = (GridSearchCV(estimator=clf_nn, 
                        param_grid=param_nn, 
                        #cv=loo,
                        cv=KFold(10, shuffle=True, random_state=seed),
                        #scoring =['neg_mean_absolute_error', 'r2'],
                        #scoring='neg_mean_squared_error',
                        scoring='r2',
                        n_jobs=-1))

print('\n1.3 Neural Network:')
print('\nhyperparameters and their ranges:')
#print(param_nn)
for i, j in param_nn.items():
    print(i[11:] + ':', j)

print('\nthe best hyperparameters:')
#gs_nn.fit(X_train, y_train)
gs_nn.fit(X, y)
#print((gs_nn.predict(X_test) - y_test))
#print(gs_nn.best_params_)
for i, j in gs_nn.best_params_.items():
    print('best ' + i[11:] + ':', j)


###################################################################
# 2. evaluate performance of different models with the best parameters
###################################################################
print('2. evaluate performance of different models with the best parameters')

print('Method: The 10-fold cv with different score functions were used to evaluate the performance.')
print('              The mean and std of scores for train and test sets were reported.')

# get the models with the best parameters
clf_best_svr = gs_svr.best_estimator_
clf_best_rf = gs_rf.best_estimator_
clf_best_nn = gs_nn.best_estimator_


# 2.1. RMSE
def rmse(y_p, y):
    return (((y_p - y) ** 2).mean())**.5

rmse_sc = metrics.make_scorer(rmse, greater_is_better=False)

print('\n2.1 Performance by RMSE:')

print('\n2.1.1 SVR:')
svr_rmse_scores = cross_validate(clf_best_svr, X, y, cv=KFold(10, shuffle=True, random_state=seed), scoring=rmse_sc, return_train_score=True)
svr_test_score = -svr_rmse_scores['test_score']
print('mean score (+/-sd) of test set: %s (+/-%s)'%(svr_test_score.mean(), svr_test_score.std()))
svr_train_score = -svr_rmse_scores['train_score']
print('mean score (+/-sd) of train set: %s (+/-%s)'%(svr_train_score.mean(), svr_train_score.std()))

print('\n2.1.2 Random forest:')
rf_rmse_scores = cross_validate(clf_best_rf, X, y, cv=KFold(10, shuffle=True, random_state=seed), scoring=rmse_sc, return_train_score=True)
rf_test_score = -rf_rmse_scores['test_score']
print('mean score (+/-sd) of test set: %s (+/-%s)'%(rf_test_score.mean(), rf_test_score.std()))
rf_train_score = -rf_rmse_scores['train_score']
print('mean score (+/-sd) of train set: %s (+/-%s)'%(rf_train_score.mean(), rf_train_score.std()))

print('\n2.1.3 Neural network:')
nn_rmse_scores = cross_validate(clf_best_nn, X, y, cv=KFold(10, shuffle=True, random_state=seed), scoring=rmse_sc, return_train_score=True)
nn_test_score = -nn_rmse_scores['test_score']
print('mean score (+/-sd) of test set: %s (+/-%s)'%(nn_test_score.mean(), nn_test_score.std()))
nn_train_score = -nn_rmse_scores['train_score']
print('mean score (+/-sd) of train set: %s (+/-%s)'%(nn_train_score.mean(), nn_train_score.std()))


# 2.2. Spearman's correlation coefficient
def scc(y_p, y):
    return spearmanr(y_p.flat, y.flat)[0]

scc_sc = metrics.make_scorer(scc, greater_is_better=True)

print('\n2.2 Performance by Spearman correlation coefficient:')

print('\n2.2.1 SVR:')
svr_scc_scores = cross_validate(clf_best_svr, X, y, cv=KFold(10, shuffle=True, random_state=seed), scoring=scc_sc, return_train_score=True)
svr_test_score = svr_scc_scores['test_score']
print('mean score (+/-sd) of test set: %s (+/-%s)'%(svr_test_score.mean(), svr_test_score.std()))
svr_train_score = svr_scc_scores['train_score']
print('mean score (+/-sd) of train set: %s (+/-%s)'%(svr_train_score.mean(), svr_train_score.std()))

print('\n2.2.2 Random forest:')
rf_scc_scores = cross_validate(clf_best_rf, X, y, cv=KFold(10, shuffle=True, random_state=seed), scoring=scc_sc, return_train_score=True)
rf_test_score = rf_scc_scores['test_score']
print('mean score (+/-sd) of test set: %s (+/-%s)'%(rf_test_score.mean(), rf_test_score.std()))
rf_train_score = rf_scc_scores['train_score']
print('mean score (+/-sd) of train set: %s (+/-%s)'%(rf_train_score.mean(), rf_train_score.std()))

print('\n2.2.3 Neural network:')
nn_scc_scores = cross_validate(clf_best_nn, X, y, cv=KFold(10, shuffle=True, random_state=seed), scoring=scc_sc, return_train_score=True)
nn_test_score = nn_scc_scores['test_score']
print('mean score (+/-sd) of test set: %s (+/-%s)'%(nn_test_score.mean(), nn_test_score.std()))
nn_train_score = nn_scc_scores['train_score']
print('mean score (+/-sd) of train set: %s (+/-%s)'%(nn_train_score.mean(), nn_train_score.std()))

# 2.3. Pearson's correlation coefficient
print('\n2.3 Performance by Pearson correlation coefficient:')

def pcc(y_p, y):
    return pearsonr(y_p.flat, y.flat)[0]

pcc_sc = metrics.make_scorer(pcc, greater_is_better=True)

print('\n2.3.1 SVR:')
svr_pcc_scores = cross_validate(clf_best_svr, X, y, cv=KFold(10, shuffle=True, random_state=seed), scoring=pcc_sc, return_train_score=True)
svr_test_score = svr_pcc_scores['test_score']
print('mean score (+/-sd) of test set: %s (+/-%s)'%(svr_test_score.mean(), svr_test_score.std()))
svr_train_score = svr_pcc_scores['train_score']
print('mean score (+/-sd) of train set: %s (+/-%s)'%(svr_train_score.mean(), svr_train_score.std()))

print('\n2.3.2 Random forest:')
rf_pcc_scores = cross_validate(clf_best_rf, X, y, cv=KFold(10, shuffle=True, random_state=seed), scoring=pcc_sc, return_train_score=True)
rf_test_score = rf_pcc_scores['test_score']
print('mean score (+/-sd) of test set: %s (+/-%s)'%(rf_test_score.mean(), rf_test_score.std()))
rf_train_score = rf_pcc_scores['train_score']
print('mean score (+/-sd) of train set: %s (+/-%s)'%(rf_train_score.mean(), rf_train_score.std()))

print('\n2.3.3 Neural network:')
nn_pcc_scores = cross_validate(clf_best_nn, X, y, cv=KFold(10, shuffle=True, random_state=seed), scoring=pcc_sc, return_train_score=True)
nn_test_score = nn_pcc_scores['test_score']
print('mean score (+/-sd) of test set: %s (+/-%s)'%(nn_test_score.mean(), nn_test_score.std()))
nn_train_score = nn_pcc_scores['train_score']
print('mean score (+/-sd) of train set: %s (+/-%s)'%(nn_train_score.mean(), nn_train_score.std()))


Writing final_project.py


In [ ]:
!python final_project.py > log_2.txt